In [1]:
import torch, pickle, shap
import numpy as np

import torch.nn as nn
import matplotlib.pyplot as plt 
import seaborn as sns 

from joblib import load, dump


nn_model = torch.load('models/shallowNN.pt')
ensemble_model = load('models/ensemble.joblib')

with open('cached/numpy_arrays.pickle', 'rb') as f:
    X_train, y_train, X_val, y_val, X_test, y_test = pickle.load(f)

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

### Customize Foward Pass


As last logistic regression layer is sklearn object, we first convert it to a complet pytorch neural network.

After testing the conversion, we convert the whole model to onnx format.

In [2]:
class Ensemble():
    
    def __init__(self):
        
        self.nn_model = torch.load('models/shallowNN.pt')
        self.ensemble_model = load('models/ensemble.joblib')
        
    def forward(self, X):
        
        X_nn = X[:,:-3]
        distance = self.nn_model.predict(X_nn)
        distance = np.expand_dims(distance, axis=1)
        X_ensemble = np.concatenate((X[:,-3:], distance),axis=1)
        return self.ensemble_model.predict_proba(X_ensemble)[:,0]
    
    
class TorchEnsemble(nn.Module):
    
    def __init__(self):
        
        super(TorchEnsemble, self).__init__()        
        self.nn_model = torch.load('models/shallowNN.pt')
        
        lr_layer = load('models/ensemble.joblib')
        
        self.ensemble_model = nn.Sequential(
            nn.Linear(in_features=4, out_features=1),
            nn.Sigmoid()
        )
        
        with torch.no_grad():
            self.ensemble_model[0].weight = nn.Parameter(torch.FloatTensor(lr_layer.coef_))
            self.ensemble_model[0].bias = nn.Parameter(torch.FloatTensor(lr_layer.intercept_))
        
    def forward(self, X):
        
        X = torch.Tensor(X)
        X1 = X[:,:-3]
        similarity = self.nn_model.forward(X1)
        similarity = torch.nn.functional.sigmoid(similarity).unsqueeze(1)
        X2 = torch.cat((X[:,-3:], similarity), dim=1)
        return 1-self.ensemble_model(X2)
    
model1 = Ensemble()
model2 = TorchEnsemble()

In [3]:
y1 = model1.forward(X_train)
y2 = model2.forward(X_train).detach().squeeze(1).numpy()

nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.


In [4]:
try:
    np.testing.assert_array_almost_equal(y1,y2, decimal=6)
    print('Test passed')
except:
    print('Test failed')

Test passed


As the test is passed, convertion to complete Pytorch network is successful. Now we convert Pytorch to ONNX and we test again:

In [5]:
torch_model = TorchEnsemble()

# Input to the model
torch_out = torch_model.forward(X_train)

# Export the model
torch.onnx.export(torch_model,               # model being run
                  torch.FloatTensor(X_train),     # model input (or a tuple for multiple inputs)
                  "models/ensemble.onnx",    # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

torch.Tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.


In [6]:
import onnxruntime

y_target = model2.forward(torch.Tensor(X_train)).detach().numpy()

ort_session = onnxruntime.InferenceSession("models/ensemble.onnx")
inputs = {'input' : X_train.astype(np.float32)}
y_onnx = ort_session.run(None, inputs)[0]

In [7]:
try:
    np.testing.assert_array_almost_equal(y_target,y_onnx, decimal=6)
    print('Test passed')
except:
    print('Test failed')

Test passed
